In [35]:
from google.colab import drive

drive.mount("/content/C")

Drive already mounted at /content/C; to attempt to forcibly remount, call drive.mount("/content/C", force_remount=True).


In [36]:
!unzip /content/C/MyDrive/CS419-IR/Cranfield.zip -d Cranfield

Archive:  /content/C/MyDrive/CS419-IR/Cranfield.zip
replace Cranfield/Cranfield/1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Cranfield/Cranfield/10.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [37]:
%%capture
!pip install nltk

In [38]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Library

In [39]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from numpy.linalg import norm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from collections import Counter,defaultdict

from tqdm import tqdm
import json
import re
import os
import math
import numpy as np
import pandas as pd
import operator
from matplotlib import pyplot as plt

from collections import OrderedDict

In [40]:
path = "/content/Cranfield/Cranfield"
path_query = "/content/Cranfield/TEST/query.txt"
path_res = "/content/Cranfield/TEST/RES/{}.txt"

# Load Data

In [41]:
# Read cranfield dataset (documents)
print("Đang đọc tài liệu Cranfield...")
documents = []
for i in tqdm(range(1, 1401), desc="Đọc tài liệu"):
    file_name = f"{i}.txt"
    path_file_name = os.path.join(path, file_name)
    with open(path_file_name, 'r', encoding='UTF-8') as file:
      documents.append(file.read())

Đang đọc tài liệu Cranfield...



Đọc tài liệu: 100%|██████████| 1400/1400 [00:00<00:00, 5087.07it/s]


In [42]:
# Read query file
raw_queries = []
with open(path_query, 'r', encoding='UTF-8') as f:
    for _ in tqdm(range(1, 226), desc="Đọc truy vấn"):
        line = f.readline()
        if not line:
            break
        raw_queries.append(line.strip())



Đọc truy vấn: 100%|██████████| 225/225 [00:00<00:00, 373601.90it/s]


In [43]:
# Read relevant documents (RES)
RES = []
for index in tqdm(range(1, 226), desc="Đọc RES"):
  with open(path_res.format(index), 'r', encoding='UTF-8') as f:
    data = f.read().split()
    ids = [int(data[i]) - 1 for i in range(1, len(data), 3)]
    RES.append(ids)



Đọc RES: 100%|██████████| 225/225 [00:00<00:00, 7130.21it/s]


In [44]:
def text_process(text):
    tokens = word_tokenize(text)
    tokens_lower = [word.lower() for word in tokens]
    cleaned_tokens = [re.sub(r'[^A-Za-z]+', '', word) for word in tokens_lower]
    cleaned_tokens = [word for word in cleaned_tokens if word]
    stop_words = set(stopwords.words('english'))
    tokens_no_stopwords = [word for word in cleaned_tokens if word not in stop_words]
    porter_stemmer = PorterStemmer()
    stemmed_tokens = [porter_stemmer.stem(word) for word in tokens_no_stopwords]
    return stemmed_tokens


# Build Vocabulary and Binary Matrix

In [45]:
# tạo từ vựng và tài liệu đã xử lý
def word_dictionary(data):
    doc = []
    unique_words_set = set()

    for text_item in tqdm(data, desc="Tiền xử lý tài liệu"):
        processed_tokens = text_process(text_item)
        doc.append(processed_tokens)
        unique_words_set.update(processed_tokens)

    word_list = sorted(list(unique_words_set))
    return doc, word_list

In [46]:
doc, word_list = word_dictionary(documents)
N = len(doc)


Tiền xử lý tài liệu: 100%|██████████| 1400/1400 [00:11<00:00, 118.84it/s]


In [47]:
len(doc)

1400

In [48]:
len(word_list)

4379

In [49]:
# chuyển tài liệu thành vector nhị phân
def documents_to_binary_vectors(doc, word_list):
    binary_vectors = []
    vocab_size = len(word_list)
    word_to_index = {word: i for i, word in enumerate(word_list)}

    for doc_tokens in tqdm(doc, desc="Vector hóa tài liệu"):
        current_doc_vector = [0] * vocab_size
        doc_terms_set = set(doc_tokens)

        for term in doc_terms_set:
            if term in word_to_index:
                index = word_to_index[term]
                current_doc_vector[index] = 1
        binary_vectors.append(current_doc_vector)
    return binary_vectors

In [50]:
doc_binary_vectors = documents_to_binary_vectors(doc, word_list)


Vector hóa tài liệu: 100%|██████████| 1400/1400 [00:00<00:00, 10166.65it/s]


# Inverted Index

In [51]:
# tạo chỉ mục nghịch đảo
def inverted_index(binary_vectors, word_list):
    index = defaultdict(list)
    num_documents = len(binary_vectors)
    vocab_size = len(word_list)

    for doc_id in tqdm(range(num_documents), desc="Build Inverted Index"):
        for term_idx in range(vocab_size):
            if binary_vectors[doc_id][term_idx] == 1:
                term = word_list[term_idx]
                index[term].append(doc_id)
    return index

In [52]:
# trích xuất DF và posting list
def extract_index_components(inverted_index, word_list):
    index_components = {}
    for term in tqdm(word_list, desc="Trích xuất thành phần"):
        posting_list = inverted_index.get(term, [])
        index_components[term] = {
            'df': len(posting_list),
            'posting_list': posting_list
        }
    return index_components

In [53]:
# lập chỉ mục cho câu truy vấn
def index_query(query_text, word_list):
    processed_query_tokens = text_process(query_text)
    word_list_set = set(word_list)
    indexed_query_terms = {term for term in processed_query_tokens if term in word_list_set}
    return indexed_query_terms

In [54]:
inverted_index = inverted_index(doc_binary_vectors, word_list)


Build Inverted Index: 100%|██████████| 1400/1400 [00:00<00:00, 1709.03it/s]


In [55]:
index_components = extract_index_components(inverted_index, word_list)


Trích xuất thành phần: 100%|██████████| 4379/4379 [00:00<00:00, 93383.04it/s]


In [56]:
print(index_components)

{'ab': {'df': 2, 'posting_list': [743, 923]}, 'abbrevi': {'df': 1, 'posting_list': [121]}, 'abil': {'df': 3, 'posting_list': [50, 76, 737]}, 'abl': {'df': 10, 'posting_list': [98, 131, 535, 580, 694, 762, 907, 913, 985, 1113]}, 'ablat': {'df': 14, 'posting_list': [81, 273, 552, 586, 1064, 1095, 1096, 1097, 1098, 1099, 1100, 1225, 1240, 1278]}, 'abrupt': {'df': 3, 'posting_list': [575, 587, 1038]}, 'abruptli': {'df': 3, 'posting_list': [438, 661, 991]}, 'absenc': {'df': 9, 'posting_list': [151, 498, 756, 965, 1146, 1236, 1280, 1320, 1322]}, 'absent': {'df': 1, 'posting_list': [627]}, 'absolut': {'df': 11, 'posting_list': [61, 413, 459, 473, 561, 986, 1002, 1009, 1034, 1260, 1394]}, 'absorb': {'df': 9, 'posting_list': [162, 163, 352, 372, 880, 1146, 1199, 1243, 1278]}, 'absorpt': {'df': 6, 'posting_list': [165, 356, 619, 1096, 1315, 1345]}, 'abstract': {'df': 2, 'posting_list': [153, 478]}, 'abundantli': {'df': 1, 'posting_list': [717]}, 'academ': {'df': 1, 'posting_list': [343]}, 'accel

# BIM Weights

In [57]:
def BIM_weights(term, N_td, N):

    effective_N_td = max(1, N_td)

    try:
        # Công thức từ tài liệu: log(0.5 * N / N_td)
        weight = math.log(0.5 * N / effective_N_td)
    except ValueError: # Handle log(0) or log(negative)
        weight = 0.0 # Assign a neutral weight
    except ZeroDivisionError: # Should be prevented by effective_N_td, but as a safeguard
        weight = 0.0

    return weight

In [58]:
def calculate_bim_score(query_terms_set, doc_tokens, N, index_components):

    score = 0.0
    doc_terms_set = set(doc_tokens)

    for term in query_terms_set:
        term_info = index_components.get(term)
        if not term_info: # Nếu từ không có trong từ điển, bỏ qua
            continue

        df_t = term_info['df'] # Document Frequency của term (N_td)

        # Tính trọng số bằng hàm BIM_weights
        w_t = BIM_weights(term, df_t, N)

        # Nếu term t có trong tài liệu, cộng trọng số của nó vào điểm
        if term in doc_terms_set:
            score += w_t

    return score

# Evaluation

In [59]:
def evaluate_model(retrieved_results, true_relevant_docs, k_values=[5, 10, 20]):
    metrics = {}
    num_relevant_retrieved = 0
    precision_at_k = {}
    sum_precisions = 0.0

    true_relevant_set = set(true_relevant_docs)
    num_true_relevant = len(true_relevant_set)

    recalls = []
    precisions = []

    for i, doc_id in enumerate(retrieved_results):
        if doc_id in true_relevant_set:
            num_relevant_retrieved += 1
            current_precision = num_relevant_retrieved / (i + 1)
            sum_precisions += current_precision

        for k in k_values:
            if (i + 1) == k:
                precision_at_k[f'P@{k}'] = num_relevant_retrieved / k

        if num_true_relevant > 0:
            recalls.append(num_relevant_retrieved / num_true_relevant)
        else:
            recalls.append(0.0)
        precisions.append(num_relevant_retrieved / (i + 1))

    total_retrieved = len(retrieved_results)
    if total_retrieved > 0:
        metrics['Precision'] = num_relevant_retrieved / total_retrieved
    else:
        metrics['Precision'] = 0.0

    if num_true_relevant > 0:
        metrics['Recall'] = num_relevant_retrieved / num_true_relevant
    else:
        metrics['Recall'] = 0.0

    if (metrics['Precision'] + metrics['Recall']) > 0:
        metrics['F1-score'] = 2 * (metrics['Precision'] * metrics['Recall']) / (metrics['Precision'] + metrics['Recall'])
    else:
        metrics['F1-score'] = 0.0

    if num_true_relevant > 0:
        metrics['AP'] = sum_precisions / num_true_relevant
    else:
        metrics['AP'] = 0.0

    metrics.update(precision_at_k)
    metrics['recalls_list'] = recalls
    metrics['precisions_list'] = precisions
    metrics['num_relevant_retrieved'] = num_relevant_retrieved
    metrics['total_retrieved_for_query'] = total_retrieved
    metrics['num_true_relevant_for_query'] = num_true_relevant
    return metrics

In [60]:
def Recall_and_Precision_r(list_re_pre):
    rs = list()
    trec = np.arange(0, 1.1, 0.1)

    for q_id in list_re_pre.keys():
        recall_list = list_re_pre[q_id]['recall']
        precision_list = list_re_pre[q_id]['precision']

        tempt = []
        for rec_level in trec:
            max_prec_at_level = 0.0
            found_at_level = False
            for i in range(len(recall_list)):
                if recall_list[i] >= rec_level:
                    max_prec_at_level = max(max_prec_at_level, np.max(precision_list[i:]))
                    found_at_level = True
                    break
            if found_at_level:
                tempt.append(max_prec_at_level)
            else:
                tempt.append(0.0)

        while len(tempt) < 11:
            tempt.append(0.0)
        rs.append(tempt)
    return rs

def MAPr(list_pre_APr):
    APr = list()
    for i in list_pre_APr:
        APr.append(np.mean(i))
    MAPr_value = np.mean(APr)
    return MAPr_value


In [61]:
for term, data in index_components.items():
    weight = BIM_weights(term, data['df'], N)
    data['weight'] = weight

In [62]:
print(index_components)

{'ab': {'df': 2, 'posting_list': [743, 923], 'weight': 5.857933154483459}, 'abbrevi': {'df': 1, 'posting_list': [121], 'weight': 6.551080335043404}, 'abil': {'df': 3, 'posting_list': [50, 76, 737], 'weight': 5.452468046375295}, 'abl': {'df': 10, 'posting_list': [98, 131, 535, 580, 694, 762, 907, 913, 985, 1113], 'weight': 4.248495242049359}, 'ablat': {'df': 14, 'posting_list': [81, 273, 552, 586, 1064, 1095, 1096, 1097, 1098, 1099, 1100, 1225, 1240, 1278], 'weight': 3.912023005428146}, 'abrupt': {'df': 3, 'posting_list': [575, 587, 1038], 'weight': 5.452468046375295}, 'abruptli': {'df': 3, 'posting_list': [438, 661, 991], 'weight': 5.452468046375295}, 'absenc': {'df': 9, 'posting_list': [151, 498, 756, 965, 1146, 1236, 1280, 1320, 1322], 'weight': 4.353855757707185}, 'absent': {'df': 1, 'posting_list': [627], 'weight': 6.551080335043404}, 'absolut': {'df': 11, 'posting_list': [61, 413, 459, 473, 561, 986, 1002, 1009, 1034, 1260, 1394], 'weight': 4.153185062245034}, 'absorb': {'df': 9, 

# Thử nghiệm

In [65]:
# Lập chỉ mục cho các truy vấn
indexed_queries = [index_query(q_text, word_list) for q_text in tqdm(raw_queries, desc="Lập chỉ mục truy vấn")]




Lập chỉ mục truy vấn:   0%|          | 0/225 [00:00<?, ?it/s]

Lập chỉ mục truy vấn: 100%|██████████| 225/225 [00:00<00:00, 1176.00it/s]


In [67]:
indexed_queries[0]

{'aeroelast',
 'aircraft',
 'construct',
 'heat',
 'high',
 'law',
 'model',
 'must',
 'obey',
 'similar',
 'speed'}

In [70]:
index_components["aeroelast"]

{'df': 18,
 'posting_list': [11,
  13,
  77,
  140,
  183,
  201,
  283,
  389,
  485,
  684,
  745,
  780,
  874,
  1065,
  1330,
  1331,
  1333,
  1360],
 'weight': 3.66070857714724}

In [72]:
candidate_docs = set()
for term in indexed_queries[0]:
    posting = index_components.get(term, {}).get('posting_list', [])
    candidate_docs.update(posting)

print(candidate_docs)

{1, 4, 5, 6, 8, 10, 11, 12, 13, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 106, 107, 109, 113, 117, 118, 119, 120, 121, 122, 123, 124, 127, 128, 130, 134, 138, 139, 140, 141, 143, 144, 145, 146, 151, 153, 155, 156, 157, 158, 159, 162, 163, 164, 165, 167, 168, 169, 171, 172, 173, 175, 176, 178, 179, 180, 182, 183, 184, 186, 188, 191, 192, 194, 196, 197, 198, 200, 201, 202, 203, 204, 205, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 224, 225, 226, 228, 229, 231, 234, 235, 236, 237, 238, 239, 241, 242, 243, 244, 247, 250, 251, 252, 254, 259, 260, 261, 262, 263, 265, 266, 267, 268, 269, 271, 273, 279, 282, 283, 284, 286, 287, 291, 292, 293, 294, 295, 296, 299, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 313, 314, 3

In [71]:
# Chạy mô hình và đánh giá

all_query_metrics = []
full_recall_precision_data = {}

total_retrieved_docs_overall = 0
total_relevant_docs_overall = 0
total_relevant_retrieved_overall = 0

for i, query_terms_set in enumerate(tqdm(indexed_queries, desc="Đánh giá truy vấn")):
    query_id = i + 1
    scores = []

    for doc_idx, doc_tokens in enumerate(doc):
        score = calculate_bim_score(
            query_terms_set,
            doc_tokens,
            N,
            index_components
        )
        scores.append((score, doc_idx))

    scores.sort(key=lambda x: x[0], reverse=True)
    retrieved_doc_ids = [doc_idx for score, doc_idx in scores]

    # Vẫn đánh giá dựa trên RES để biết hiệu suất của mô hình trong trường hợp này
    query_metrics = evaluate_model(retrieved_doc_ids, RES[i])
    all_query_metrics.append(query_metrics)

    total_retrieved_docs_overall += query_metrics['total_retrieved_for_query']
    total_relevant_docs_overall += query_metrics['num_true_relevant_for_query']
    total_relevant_retrieved_overall += query_metrics['num_relevant_retrieved']

    full_recall_precision_data[query_id] = {
        'recall': query_metrics['recalls_list'],
        'precision': query_metrics['precisions_list']
    }

# --- Tính toán Mean Average Precision (MAP) và các chỉ số trung bình khác ---
total_precision = 0.0
total_recall = 0.0
total_f1 = 0.0
total_ap = 0.0

num_queries_evaluated = len(all_query_metrics)

for metrics in all_query_metrics:
    total_precision += metrics.get('Precision', 0)
    total_recall += metrics.get('Recall', 0)
    total_f1 += metrics.get('F1-score', 0)
    total_ap += metrics.get('AP', 0)

print("\n--- Kết quả Đánh giá Mô hình BIM (Không có ngữ liệu mẫu) ---")
if num_queries_evaluated > 0:
    print(f"Precision Trung bình: {total_precision / num_queries_evaluated:.4f}")
    print(f"Recall Trung bình: {total_recall / num_queries_evaluated:.4f}")
    print(f"F1-score Trung bình: {total_f1 / num_queries_evaluated:.4f}")
    print(f"Mean Average Precision (MAP): {total_ap / num_queries_evaluated:.4f}")

else:
    print("Không có truy vấn nào được đánh giá.")

# --- Tính toán MAP nội suy (MAPr) ---
if num_queries_evaluated > 0:
    avg_recall_precision_r_data = Recall_and_Precision_r(full_recall_precision_data)
    map_r_value = MAPr(avg_recall_precision_r_data)
    print(f"Mean Average Precision (Interpolated) (MAPr): {map_r_value:.4f}")
else:
    print("Không thể tính MAPr do không có truy vấn nào được đánh giá.")

# --- In thêm các kết quả yêu cầu ---
print("\n--- Thống kê tổng quan ---")
print(f"Số lượng term duy nhất được sử dụng (kích thước từ vựng): {len(word_list)}")
print(f"Tổng số tài liệu trong bộ sưu tập (N): {N}")
print(f"Tổng số truy vấn: {len(raw_queries)}")
print(f"Tổng số tài liệu trả về (tất cả truy vấn): {total_retrieved_docs_overall}")
print(f"Tổng số tài liệu liên quan thực sự (tất cả truy vấn): {total_relevant_docs_overall}")
total_non_relevant_retrieved_overall = total_retrieved_docs_overall - total_relevant_retrieved_overall
print(f"Tổng số tài liệu không liên quan được trả về (tất cả truy vấn): {total_non_relevant_retrieved_overall}")


Đánh giá truy vấn: 100%|██████████| 225/225 [00:05<00:00, 44.63it/s]



--- Kết quả Đánh giá Mô hình BIM (Không có ngữ liệu mẫu) ---
Precision Trung bình: 0.0058
Recall Trung bình: 0.9995
F1-score Trung bình: 0.0115
Mean Average Precision (MAP): 0.3113
Mean Average Precision (Interpolated) (MAPr): 0.3320

--- Thống kê tổng quan ---
Số lượng term duy nhất được sử dụng (kích thước từ vựng): 4379
Tổng số tài liệu trong bộ sưu tập (N): 1400
Tổng số truy vấn: 225
Tổng số tài liệu trả về (tất cả truy vấn): 315000
Tổng số tài liệu liên quan thực sự (tất cả truy vấn): 1833
Tổng số tài liệu không liên quan được trả về (tất cả truy vấn): 313168


In [73]:
print("\nCác tài liệu ứng viên cho indexed_queries[0]:")
candidate_docs = set()
term_weights = {}

target_query_terms = indexed_queries[0]
for term in target_query_terms:
    term_info = index_components.get(term)
    if term_info:
        weight = term_info['weight']
        for doc_id in term_info['posting_list']:
            if doc_id not in term_weights:
                term_weights[doc_id] = 0.0
            term_weights[doc_id] += weight

sorted_doc_weights = sorted(term_weights.items(), key=lambda x: x[1], reverse=True)

print("\nKết quả truy xuất cho truy vấn đầu tiên (doc_id: tổng trọng số):")
for doc_id, score in sorted_doc_weights:
    print(f"{doc_id}: {score:.4f}")


Các tài liệu ứng viên cho indexed_queries[0]:

Kết quả truy xuất cho truy vấn đầu tiên (doc_id: tổng trọng số):
572: 12.8161
485: 11.9526
328: 11.6864
50: 9.7498
13: 9.2856
183: 8.8712
11: 8.7382
877: 8.4624
575: 8.3704
943: 8.3499
77: 8.1984
664: 8.0527
201: 7.3240
452: 7.2906
1002: 7.1548
1360: 7.1475
140: 6.9972
171: 6.9092
1267: 6.8233
218: 6.7872
639: 6.5840
728: 6.5008
745: 6.4498
1193: 6.3166
662: 6.0895
1334: 6.0675
373: 6.0349
684: 5.9100
916: 5.8220
1245: 5.7971
413: 5.7691
1262: 5.7111
194: 5.6249
746: 5.6249
791: 5.6249
1146: 5.4483
1033: 5.4183
600: 5.2997
164: 5.2400
219: 5.1259
718: 5.0868
878: 5.0868
1337: 5.0868
24: 5.0807
327: 5.0775
363: 5.0775
1299: 5.0775
874: 5.0356
1065: 5.0356
12: 4.9554
159: 4.9554
304: 4.9554
331: 4.9554
1071: 4.9554
1238: 4.9477
720: 4.9228
251: 4.8836
292: 4.8836
314: 4.8836
571: 4.8368
215: 4.7991
528: 4.7991
624: 4.7832
1197: 4.7832
907: 4.7506
1143: 4.7506
810: 4.7099
1: 4.6208
587: 4.6208
1312: 4.5740
310: 4.5377
780: 4.5350
28: 4.4908
